### import statements

In [ ]:
%matplotlib inline
import sys
import os
import pandas as pd
sys.path.append('../src')
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import gaussian_kde
import seaborn as sns
import scipy.stats as stats
from mpl_toolkits.axes_grid1 import make_axes_locatable

### constants

In [ ]:
F_INPUT = '../../data/v2/data.h5'

### read data

In [ ]:
df = pd.read_hdf(F_INPUT, 'merged')

### add new features

In [ ]:
df['day'] = df.index.dayofyear
df['logCn2'] = np.log10(df['Cn2'])

In [ ]:
feats = ['pressure', 'relative_humidity', 'temperature', 'wind_speed', 'logCn2', 'solar_zenith_angle','day']
label_day = 'r0_day'
label = 'r0'
label_night = 'r0_night'
feats_plus_r0 = feats + ['r0']
feats_plus_r0day = feats + ['r0_day']
feats_plus_r0night = feats + ['r0_night']

### restricting data to usable, relatively dense subset

In [ ]:
df_subset = df[(df.index > '2018-05-03') & (df.index < '2020-12-30')]

In [ ]:
df_subset.describe()

### finding non-nan values

In [ ]:
valid = ~df_subset[feats_plus_r0].isnull().any(axis=1)

In [ ]:
df_subset.loc[valid, feats_plus_r0].count()

In [ ]:
valid_day = ~df_subset[feats_plus_r0day].isnull().any(axis=1)

In [ ]:
df_subset.loc[valid_day, feats_plus_r0day].count()

In [ ]:
valid_night = ~df_subset[feats_plus_r0night].isnull().any(axis=1)

In [ ]:
df_subset.loc[valid_night,feats_plus_r0night].count()

### splitting into train and test

In [ ]:
split_date = '2019-12-31'
train = df_subset.index <= split_date
test  = df_subset.index > split_date

In [ ]:
test_truth_night = df_subset.loc[test&valid_night,label_night]
test_truth_day = df_subset.loc[test&valid_day,label_day]
test_truth_all = df_subset.loc[test&valid,label]

In [ ]:
df_subset.loc[train&valid,feats_plus_r0].count()

In [ ]:
df_subset.loc[test&valid,feats_plus_r0].count()

In [ ]:
df_subset.loc[train&valid_day,feats_plus_r0day].count()

In [ ]:
df_subset.loc[test&valid_day,feats_plus_r0day].count()

In [ ]:
df_subset.loc[train&valid_night,feats_plus_r0night].count()

In [ ]:
df_subset.loc[test&valid_night,feats_plus_r0night].count()

### initializing the RF regressor

In [ ]:
regr = RandomForestRegressor(n_estimators=100, random_state=0)

### train and test subroutine

In [ ]:
def train_and_test(train_df, test_df, feats, label):
    regr.fit(train_df[feats], train_df[label])
    r2 = regr.score(test_df[feats], test_df[label])
    preds = regr.predict(test_df[feats])
    return preds, r2

### evaluation

In [ ]:
test_preds_all, test_r2_all = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats, label)
test_r2_all

In [ ]:
def error_diff(targ, pred):
    return targ-pred
def error_perc(targ, pred):
    return (targ-pred)/targ

### plots

In [ ]:
def density_estimation(m1, m2, xmin, xmax, ymin, ymax):
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]                                                     
    positions = np.vstack([X.ravel(), Y.ravel()])                                                       
    values = np.vstack([m1, m2])                                                                        
    kernel = stats.gaussian_kde(values)                                                                 
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z

def scatter_with_errors(truth, preds, error_func, xmin, xmax, ymin, ymax):
    fig, ax = plt.subplots(2, 2, figsize=(20, 20))
    s = 25
    a = 0.4
    ax[0,0].scatter(truth, preds, edgecolor='k', c="cornflowerblue", s=s, alpha=a)
    x = np.linspace(truth.min(), truth.max(), 1000)
    ax[0,0].plot(x, x, 'r-')
    ax[0,0].set_xlabel("Actual r0")
    ax[0,0].set_ylabel("Predicted r0")
    ax[0,0].set_xlim([xmin, xmax])                                                                           
    ax[0,0].set_ylim([ymin, ymax]) 

    
    X, Y, Z = density_estimation(truth, preds, xmin, xmax, ymin, ymax)
    ax[0,1].plot(x, x, 'r-')
    im01 = ax[0,1].contourf(X, Y, Z, 20)
    divider01 = make_axes_locatable(ax[0,1])
    cax01 = divider01.append_axes('right', size='5%', pad=0.05)
    ax[0,1].set_xlim([xmin, xmax])                                                                           
    ax[0,1].set_ylim([ymin, ymax]) 
    fig.colorbar(im01, cax=cax01)


    # now plot with errors
    #
    errs = error_func(truth, preds)
    ymin = np.min(errs) - 1
    ymax = np.max(errs) + 1
    
    
    ax[1,0].scatter(truth, errs, edgecolor='k', c="forestgreen", s=s, alpha=a)
    ax[1,0].plot(x, np.zeros(x.shape), 'r-')
    ax[1,0].set_xlabel("Actual r0")
    ax[1,0].set_ylabel("Error r0")
    ax[1,0].set_xlim([xmin, xmax])     
    ax[1,0].set_ylim([ymin, ymax])                                                                           

    
    Xerr, Yerr, Zerr = density_estimation(truth, errs, xmin, xmax, ymin, ymax)
    ax[1,1].plot(x, np.zeros(x.shape), 'r-')
    im11 = ax[1,1].contourf(Xerr, Yerr, Zerr, 20)
    divider11 = make_axes_locatable(ax[1,1])
    cax11 = divider11.append_axes('right', size='5%', pad=0.05)
    ax[1,1].set_xlim([xmin, xmax]) 
    ax[1,1].set_ylim([ymin, ymax])                                                                           
    fig.colorbar(im11, cax=cax11)

    plt.show()

#### scatter plots of actual vs. predict using error_diff

In [ ]:
%matplotlib inline
xmin, ymin = 0, 0
xmax, ymax = 20, 20
scatter_with_errors(test_truth_all, test_preds_all, error_perc, xmin, xmax, ymin, ymax)

#### interactive time domain plot of errors

switching matplotlib to notebook mode to enable a zoom-in of different portions of the time axis

In [ ]:
def plot_errors_in_time(truth, preds):
    fig, ax = plt.subplots(3, 1)
    ax[0].plot(truth.index, truth, 'gx', label='actual')
    ax[0].plot(truth.index, preds, 'ro', label='predicted')
    ax[0].set_xlabel("Datetime")
    ax[0].set_ylabel("r0")
    ax[0].legend()

    ax[1].plot(truth.index, error_diff(truth, preds), 'bx')
    ax[1].set_xlabel("Datetime")
    ax[1].set_ylabel("error r0")

    ax[2].plot(truth.index, error_perc(truth, preds), 'bx')
    ax[2].set_xlabel("Datetime")
    ax[2].set_ylabel("perc error r0")
    plt.show()

In [ ]:
%matplotlib notebook 
plot_errors_in_time(test_truth_all, test_preds_all)

#### feature importance

feature importance from the model

In [ ]:
%matplotlib inline
def plot_importance(forest, X, featnames):
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print(f"{f + 1}. {featnames[indices[f]]:20} ({importances[indices[f]]})")

    # Plot the impurity-based feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
            color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), [ featnames[i] for i in indices ], rotation='vertical')
    plt.xlim([-1, X.shape[1]])
    plt.show()

In [ ]:
plot_importance(regr, df_subset.loc[train&valid,feats], feats)

## debug why CN2 is so low

#### What happens if I drop month and SZA

Answer: turns out we had to take the log of CN2

In [ ]:
feats_no_sza = ['pressure', 'relative_humidity', 'temperature', 'wind_speed', 'logCn2']
preds_all_no_sza, r2_all_no_sza = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats_no_sza, label)
scatter_with_errors(test_truth_all, preds_all_no_sza, error_perc, xmin, xmax, ymin, ymax)
plot_importance(regr, df_subset.loc[train&valid,feats_no_sza], feats_no_sza)
r2_all_no_sza

#### correlation between the signals using [stats.pearsonr](https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9)

We calculate:
- overall synchrony between r0 and Cn2
- local synchrony between r0 and Cn2

#### Overall Synchrony

In [ ]:
def plot_overall_synchrony(feat1, feat2, feat1name, feat2name, r):
    f,ax=plt.subplots(2, 1, figsize=(7,3), sharex=True)
    ax[0].plot(feat1, label=feat1name)
    ax[1].plot(feat2, label=feat2name)
    ax[1].set(title=f"Overall Pearson r = {np.round(r,2)}");
    return

def plot_local_synchrony(feat1, feat2, feat1name, feat2name, r_window_size=120):
    # Compute rolling window synchrony
    rolling_r = feat1.rolling(window=r_window_size, center=True).corr(feat2)
    f,ax=plt.subplots(3,1,figsize=(14,6),sharex=True)
    ax[0].plot(feat1, label=feat1name)
    ax[1].plot(feat2, label=feat2name)
    rolling_r.plot(ax=ax[2])
    ax[0].set(ylabel=feat1name)
    ax[1].set(ylabel=feat2name)
    ax[2].set(ylabel='Pearson r')
    plt.suptitle("data and rolling window correlation")
    
    
def print_pearsonr(feat1, feat2):

    r, p = stats.pearsonr(feat1, feat2)
    print(f"Scipy computed Pearson r: {r} and p-value: {p}")
    return r, p

In [ ]:
r, p = print_pearsonr(df_subset.loc[train&valid,label], df_subset.loc[train&valid,'logCn2'])
# plot_overall_synchrony(df_subset.loc[train&valid,label], df_subset.loc[train&valid,['logCn2']], label, 'logCn2', r)

#### Local Synchrony

In [ ]:
plot_local_synchrony(df_subset.loc[train&valid,label], df_subset.loc[train&valid,['logCn2']], label, 'Cn2')

## Synchrony using only R0 daytime data

#### Overall Synchrony

In [ ]:
r, p = print_pearsonr(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,'logCn2'])
r

In [ ]:
# plot_overall_synchrony(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,['logCn2']], label_day, 'logCn2', r)

#### Local Synchrony

In [ ]:
plot_local_synchrony(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,['logCn2']], label_day, 'logCn2')

#### Classifier Using R0 daytime data

In [ ]:
test_preds_day, test_r2_day = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats, label_day)
scatter_with_errors(test_truth_day, test_preds_day, error_perc, xmin, xmax, ymin, ymax)
plot_importance(regr, df_subset.loc[train&valid_day,feats], feats)
test_r2_day

#### Classifier with Night-time r0 only

In [ ]:
test_preds_night, test_r2_night = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats, label_night)
scatter_with_errors(test_truth_night, test_preds_night, error_perc, xmin, xmax, ymin, ymax)
plot_importance(regr, df_subset.loc[train&valid_night,feats], feats)
test_r2_night

## Histograms by Magnitude

In [ ]:
def r0_histograms(r0, r0min, r0max, key):
    bins = np.linspace(r0min, r0max, 100)
    plt.yscale('log')
    plt.hist(r0, bins, histtype='step', label=key)
    
r0_histograms(test_truth_day, 0, 80, 'r0 day')
r0_histograms(test_truth_night, 0, 80, 'r0 night')
plt.legend()
plt.show()

## Performance Histograms by Magnitude

In [ ]:
def error_by_r0_histograms(truth, errs, r0min, r0max):
    plt.figure()
    bins = np.linspace(r0min, r0max, 81)
    d = {'r0_floor': np.floor(truth), 'errs': errs}
    df = pd.DataFrame(data=d)
    errs_by_r0_mean = df.groupby('r0_floor').mean()
    errs_by_r0_std = df.groupby('r0_floor').std()
    x = sorted(df['r0_floor'].unique())
    print(np.arange(0, max(x), 5))
    plt.bar(x, errs_by_r0_mean['errs'], yerr=errs_by_r0_std['errs'])
    plt.xticks(np.arange(0, max(x), 4))

    plt.xlabel('r0 floor')
    plt.ylabel('perc error')
    plt.show()
    return


error_by_r0_histograms(test_truth_all, error_perc(test_truth_all, test_preds_all), 0, 80)
error_by_r0_histograms(test_truth_day, error_perc(test_truth_day, test_preds_day), 0, 80)
error_by_r0_histograms(test_truth_night, error_perc(test_truth_night, test_preds_night), 0, 80)